In [13]:
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd
from sklearn.neighbors import NearestNeighbors

import re
import faiss
import torch

In [7]:
#pip install faiss-gpu

     |████████████████████████████████| 85.5 MB 125.1 MB/s eta 0:00:01  |█▍                              | 3.8 MB 6.9 MB/s eta 0:00:12
Note: you may need to restart the kernel to use updated packages.


In [9]:
pip install faiss-cpu

     |████████████████████████████████| 27.0 MB 7.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [14]:
model_path = '/home/u4026/KoSentenceBERT_SKTBERT/output/training_sts'
model = SentenceTransformer(model_path)
#model = SentenceTransformer("distiluse-base-multilingual-cased-v1")

using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_v1.zip
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
using cached model. /home/u4026/CBNU_KoSentenceBERT_SKT/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece
Load Model


/home/u4026/.local/lib/python3.8/site-packages/sentence_transformers/models/Transformer.py:41: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self.auto_model.load_state_dict(

In [4]:
#pip show transformers

Name: transformers
Version: 4.46.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: /home/u4026/.local/lib/python3.8/site-packages
Requires: regex, filelock, pyyaml, tokenizers, huggingface-hub, safetensors, requests, numpy, tqdm, packaging
Required-by: sentence-transformers, kobert
Note: you may need to restart the kernel to use updated packages.


In [15]:
# CSV 파일 로드
csv_file = "books_summary.csv"
data = pd.read_csv(csv_file)

In [25]:
def split_and_filter_sentences(text):               #무시하자
    """
    텍스트를 KSS와 추가 기준으로 분리하고, 의미 없는 연결어를 제거
    """
    # KSS로 1차 분리
    sentences = kss.split_sentences(text)
    
    # 추가 기준 적용 ('그리고', '또', ',' 기준으로 분리)
    split_sentences = []
    for sentence in sentences:
        parts = re.split(r'(그리고|또|,)', sentence)
        parts = [p.strip() for p in parts if p.strip()]  # 공백 제거 및 빈 문자열 제거
        split_sentences.extend(parts)
    
    # 불필요한 연결어 필터링
    filtered_sentences = [s for s in split_sentences if s not in ["그리고", "또", ","]]
    
    return filtered_sentences

In [21]:
def get_average_embedding(sentences, model):        #무시하자
    """
    여러 문장의 임베딩 값을 평균화
    """
    if not sentences:
        return None  # 빈 입력에 대한 처리

    embeddings = model.encode(sentences, convert_to_tensor=True)
    avg_embedding = embeddings.mean(dim=0)  # 평균 계산
    return avg_embedding

In [17]:
# 전처리된 텍스트 열 추출
corpus = data['내용요약'].tolist()
titles = data['첵제목'].tolist()

In [26]:
# 코퍼스 문장 분리 및 임베딩 평균 계산                무시하자
corpus_embeddings = []
for doc in corpus:
    sentences = split_and_filter_sentences(doc)
    avg_embedding = get_average_embedding(sentences, model)
    corpus_embeddings.append(avg_embedding)

In [19]:
# STS 코퍼스 임베딩 생성
#sts_corpus_embeddings = model.encode(corpus, convert_to_tensor=True)

In [8]:
embedding_dimension = corpus_embeddings.shape[1]

In [27]:
sts_embedding_dimension = sts_corpus_embeddings.shape[1]

In [ ]:
print("상위 5개 '내용요약' 데이터:")
print(corpus[:5]) 

In [28]:
# FAISS CPU 설정
index = faiss.IndexFlatL2(embedding_dimension)  # L2 거리 기반 인덱스 생성
index.add(sts_corpus_embeddings.cpu().numpy())

In [21]:
# 후보군 찾기 (KNN 활용)
def find_candidates_with_knn(query, corpus, titles, n_neighbors=3):
    # STS 임베딩 생성
    corpus_embeddings = model.encode(corpus)
    query_embedding = model.encode([query])

    # Nearest Neighbors 모델 초기화 및 학습
    knn = NearestNeighbors(n_neighbors=n_neighbors, metric='cosine')
    knn.fit(corpus_embeddings)

    # Query와 가장 가까운 이웃 찾기
    distances, indices = knn.kneighbors(query_embedding)

    candidates = []
    for dist, idx in zip(distances[0], indices[0]):
        candidates.append({
            "책제목": titles[idx],
            "내용요약": corpus[idx],
            "유사도": 1 - dist  # 코사인 거리 -> 유사도로 변환
        })

    # 유사도를 기준으로 정렬
    return sorted(candidates, key=lambda x: x["유사도"], reverse=True)

In [22]:
# sts를 활용해 포함 관계 분석
def find_entailments_with_batches(query, corpus, titles, batch_size=500, top_k=3, threshold=0.2):
    results = []
    query_embedding = model.encode([query], convert_to_tensor=True)

    for i in range(0, len(corpus), batch_size):
        batch_corpus = corpus[i:i + batch_size]
        batch_titles = titles[i:i + batch_size]
        batch_embeddings = model.encode(batch_corpus, convert_to_tensor=True)

        # 코사인 유사도 계산 (NLI 모델 활용)
        similarities = util.pytorch_cos_sim(query_embedding, batch_embeddings)

        for j, similarity in enumerate(similarities[0]):
            score = similarity.item()
            if score >= threshold:  # Threshold 적용
                results.append({
                    "책제목": batch_titles[j],
                    "내용요약": batch_corpus[j],
                    "포함확률": score
                })

    # 상위 N개 결과 정렬
    return sorted(results, key=lambda x: x["포함확률"], reverse=True)[:top_k]


In [29]:
#faiss 써서 포함관계분석
def find_entailments_with_faiss(query, corpus, titles, top_k=3, threshold=1.0):
    results = []
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()  # GPU 텐서를 CPU NumPy 배열로 변환

    # FAISS에서 최근접 이웃 검색
    distances, indices = index.search(query_embedding, top_k)

    for dist, idx in zip(distances[0], indices[0]):
        score = 1 - dist  # 거리 값을 유사도로 변환
        if score <=threshold:  # Threshold 적용
            results.append({
                "책제목": titles[idx],
                "내용요약": corpus[idx],
                 "포함확률": 1 - dist if dist < 1 else 0,  # 거리 값을 유사도로 변환
                "L2 거리": dist
                #"포함확률": score
            })

    return sorted(results, key=lambda x: x["포함확률"], reverse=True)[:top_k]

In [19]:
# NLI모델로 평가
def evaluate_with_nli(query, entailment_results):
    query_embedding = nli_model.encode([query], convert_to_tensor=True)
    results = []

    for result in entailment_results:
        corpus_embedding = nli_model.encode([result['내용요약']], convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(query_embedding, corpus_embedding).item()
        results.append({
            "책제목": result['책제목'],
            "내용요약": result['내용요약'],
            "포함확률": result['포함확률'],
            "유사도": similarity
        })

    # 유사도를 기준으로 정렬
    return sorted(results, key=lambda x: x["유사도"], reverse=True)

In [11]:
# STS 모델로 평가
def evaluate_with_sts(query, entailment_results):
    query_embedding = model.encode([query], convert_to_tensor=True)
    results = []

    for result in entailment_results:
        corpus_embedding = model.encode([result['내용요약']], convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(query_embedding, corpus_embedding).item()
        results.append({
            "책제목": result['책제목'],
            "내용요약": result['내용요약'],
            "포함확률": result['포함확률'],
            "유사도": similarity
        })

    # 유사도를 기준으로 정렬
    return sorted(results, key=lambda x: x["유사도"], reverse=True)

In [14]:
# Step 2: 쿼리 입력 및 상위 3개 검색           이거안씀
def search_nearest_neighbors(query, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().numpy()  # GPU 텐서를 NumPy 배열로 변환
    
    # FAISS에서 상위 3개 검색
    distances, indices = index.search(query_embedding, top_k)
    
    # 검색된 3개의 데이터 반환
    nearest_data = [{"index": idx, "내용요약": corpus[idx], "distance": dist}
                    for dist, idx in zip(distances[0], indices[0])]
    return nearest_data


In [16]:
# Step 3: 검색된 데이터와 쿼리 상세 비교       이거 안씀
def compare_with_nearest(query, nearest_data):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu()  # GPU 텐서를 CPU 텐서로 변환
    
    # 검색된 데이터 임베딩 추출
    selected_embeddings = np.array([corpus_embeddings[entry["index"]].cpu().numpy() for entry in nearest_data])
    
    # 유사도 계산 (코사인 유사도)
    similarities = util.pytorch_cos_sim(query_embedding, selected_embeddings)
    
    # 유사도 추가
    for i, entry in enumerate(nearest_data):
        entry["유사도"] = similarities[0][i].item()
    return sorted(nearest_data, key=lambda x: x["유사도"], reverse=True)

In [30]:
# 최종 유사도 평가 (STS 활용)
def evaluate_with_stsk(query, candidates):
    query_embedding = model.encode([query], convert_to_tensor=True)
    results = []

    for candidate in candidates:
        corpus_embedding = model.encode([candidate['내용요약']], convert_to_tensor=True)
        similarity = util.pytorch_cos_sim(query_embedding, corpus_embedding).item()

        results.append({
            "책제목": candidate['책제목'],
            "내용요약": candidate['내용요약'],
            "유사도": similarity
        })

    # 유사도를 기준으로 정렬
    return sorted(results, key=lambda x: x["유사도"], reverse=True)

In [23]:
# 사용자가 입력한 텍스트
query = input("텍스트를 입력하세요: ")


# 쿼리 임베딩 생성
#query_embedding = nli_model.encode([query], convert_to_tensor=True)

텍스트를 입력하세요: 새 교육과정과 관련된 책 추천해줘


In [31]:
# NLI를 사용해 상위 3개 필터링
#entailment_results = find_entailments_with_batches(query, corpus, titles, batch_size=500, top_k=3)
#knn
candidates= find_candidates_with_knn(query, corpus, titles, n_neighbors=3)

# faiss를 사용해 상위 3개 필터링
#entailment_results = find_entailments_with_faiss(query,corpus,titles, top_k=3)

# NLI로 유사도 평가
#final_results = evaluate_with_nli(query, entailment_results)

#STS로 유사도 평가
final_results = evaluate_with_stsk(query, entailment_results)

# 결과 출력
print("\n가장 관련성 높은 책 추천:")
for result in final_results:
    print(f"책제목: {result['책제목']}")
    print(f"내용요약: {result['내용요약']}")
   # print(f"포함확률: {result['포함확률']:.4f}")
    print(f"유사도: {result['유사도']:.4f}\n")



가장 관련성 높은 책 추천:
책제목: 교육광장 2016 겨울호 (Vol. 62)
내용요약: 새 교육과정에 따른 평가기준의 개발이 필요해서, 한국교육과정평가원에서는 총론과 교과별 평가기준 등을 연구 개발 하였다.  핵심 개발 방향은 2015년 개정 교육과정이 학교현장에서 운영되도록 하는 것이다.
유사도: 0.5778

책제목: 교육정책포럼 2015년 07월 (통권 265호)
내용요약: 개정 교육과정의 운영 방법을 명확히 하기 위해 전반적으로 학생들의 학습 강점과 요구사항 등을 교육과정에 적절히 반영하였다.
유사도: 0.5211

책제목: 미래 사회 교육 환경 변화에 따른 교과서 발전 방안
내용요약: 함수곤은 전통적인 학습 교재로서의 교과서의 역할을 다섯 가지 제시했다. 또한 학습 과제의 효율적인 제시, 학습의 개성화 기능, 정착 기능 등을 최근 등장한 교과서의 학습재로서의 역할을 위해 필요하다고 설명했다.
유사도: 0.4998



In [22]:
# 함수 호출           이거 안씀
nearest_data = search_nearest_neighbors(query, top_k=3)
final_results = compare_with_nearest(query, nearest_data)

# 결과 출력
print("\n가장 유사한 3개의 문장:")
for idx, result in enumerate(final_results):
    print(f"Rank {idx + 1}:")
    print(f"내용요약: {result['내용요약']}")
    print(f"유사도: {result['유사도']:.4f}\n")


가장 유사한 3개의 문장:
Rank 1:
내용요약: 연구반은 지역 MBC, KBS 지역총국의 편성‧경영 현황을 검토하고 관계자의 자체제작물 편성의무 의견과 논의를 통해 제도적 실현가능성과 세부사항을 논의했다. 
유사도: 0.5576

Rank 2:
내용요약: PD는 자격유지를 위해 기획재정부 고시 규정상 의무를 이행해야하는 바, 이행 실적에 따라 점수로 평가를 받게되고, PPD는 특성상 보다 적은 의무를 부과한다.
유사도: 0.5176

Rank 3:
내용요약: 한국산업기술평가관리원은 국가 혁신역량제고를 목적으로 설립되었다. 기관장은 설립목적에 부합하는 다양한 경영방침을 수립하고 신핵심가치 설정과 이를 실행하기 위한 내부 실행력을 강화하고 공공기관의 사회적 책임을 다하기 위해 노력하는 등의 모습을 보였다.
유사도: 0.4974



In [28]:
# 코사인 유사도 비교         #이거 안씀
if query_embedding is not None:
    cos_scores = [util.pytorch_cos_sim(query_embedding, corpus_embedding)[0].item() for corpus_embedding in corpus_embeddings]

    # 상위 유사도 결과 출력
    top_k = min(5, len(cos_scores))
    top_results = np.argsort(cos_scores)[-top_k:][::-1]  # 유사도 높은 순으로 정렬

    print("\n\n======================\n\n")
    print("Query:", query)
    print("\nTop {} most similar sentences in corpus:".format(top_k))

    for idx in top_results:
        print(f"{corpus[0]} (Score: {cos_scores[idx]:.4f})")
else:
    print("입력 텍스트에서 유효한 문장이 없습니다.")





Query: 요즘 우리나라 대통령이 개엄령을 일으켜서 놀라웠는데 예전에 있었던 광주시위에 관한책을 소개시켜줘 광주시위후 슬픔을 받는 사람들의 이야기였으면 해

Top 3 most similar sentences in corpus:
소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다 (Score: 0.4742)
소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다 (Score: 0.2166)
소설은 6개의 장으로 나누어져 있었는데요 각 장마다 다른 인물의 시선으로 이야기가 진행되고 있었어요 주인공들은 광주에서 일어난 시위 사건 속에 있던 사람들로 각자의 슬픔을 안고 살아가고 있습니다 (Score: 0.1448)


In [2]:
# 코사인 유사도 계산
cos_scores = util.pytorch_cos_sim(query_embedding, corpus_embeddings)[0]
cos_scores = cos_scores.cpu()

# 책 제목별 최고 유사도 선택
best_results = {}
for idx, score in enumerate(cos_scores):
    title = titles[idx]
    sentence = corpus[idx].strip()
    if title not in best_results or best_results[title]['score'] < score.item():
        best_results[title] = {'sentence': sentence, 'score': score.item()}

    # 유사도 점수가 높은 순으로 정렬
sorted_results = sorted(best_results.items(), key=lambda x: x[1]['score'], reverse=True)

# 가장 유사한 결과 가져오기
#top_k = 3
#top_results = np.argpartition(-cos_scores, range(top_k))[0:top_k]

# 결과 출력
print("\n\n======================\n\n")
print("Query:", query)
print("\nTop 5 most similar sentences in corpus:")
for title, result in sorted_results[:top_k]:
        print(f"Title: {title}, Sentence: {result['sentence']} (Score: {result['score']:.4f})")

#for idx in top_results[0:top_k]:
    #print(f"Title: {titles[idx]}, Sentence: {corpus[idx].strip()} (Score: {cos_scores[idx]:.4f})")
    #print(f"{corpus[idx].strip()} (Score: {cos_scores[idx]:.4f})")

NameError: name 'util' is not defined